## Analiza sentimenata IMDb skupa podataka korišćenjem rekurentnih neuronskih mreža



In [1]:
#!pip install tensorflow

Potrebno je izvršiti prethodnu liniju koda ako nemamo instaliran tensorflow na racunaru.

In [2]:
import numpy as np


import tensorflow as tf
from tensorflow import keras

print("Version: ", tf.__version__)


Version:  2.15.0


Importovani su svi moduli koji će se koristiti u ovom projektu. Verzija TensorFlow-a koja se korsiti je 2.15.0 čime potvrđujemo uspešnost instalacije.

In [3]:
imdb = keras.datasets.imdb #učitavamo skup podataka

print(type(imdb))
vocabulary =imdb.get_word_index() #učitavamo rečnik
NUM_WORDS = len(vocabulary) #definišemo promenljivu broj reči u skupu podataka koja će nam kasnije biti potrebna

print(NUM_WORDS)


<class 'module'>
88584


Preko keras.dataset je preuzet skup podataka koji će se koristiti.

IMDb skup podataka sadrži 25,000 recenzija za testiranje i 25,000 za treniranje koje su već šifrovane i označene kao pozitivne ili negativne. Svaka recenzija je šifrovana tako da broj označava koliko često se reč javlja u celom skupu podataka. Ako je reč šifrovana sa brojem 5 to znači da je 5. najčešća reč u skupu podataka. Skup je balansiran tako da sadrži podjednako pozitivnih i negativnih recenzija. Negativna recenzija znači da je ocena filma manja ili jednaka 4 od 10, a pozitivna recenzija ima ocenu veću ili jednaku 7 od 10. Neutralnije recenzije nisu obuhvaćene ovim skupom. Pozitivne su šifrovane kao 1, a negativne kao 0.

Kao što vidimo, imdb je modul koji sadrži dve funkcije: get_word_index() i load_data(). Preko get_word_index() dobijamo mapirani rečnik koji je sačuvan u promenljivoj vocabulary. U skupu podataka imamo 88584 reči koje će ući u analizu.

In [4]:
(train_dataset, train_labels), (test_dataset, test_labels) = imdb.load_data(num_words = NUM_WORDS)

Preko funkcije load_data() učitavamo IMDb skup podataka. Parametar num_words određuje koliko će najčešće korišćenih reči da se zadrži u modelu, ovaj parametar je jako koristan kada radimo sa velikim skupovima podataka jer može značajno smanjiti vreme treniranja i obrade podataka. U ovom radu se zadržavaju sve reči iz rečnika 'vocabulary'. 

IMDb skup podataka sadrži fajlove sa podacima za trening i test set kao i zasebne fajlove za njihove zavisne promenljive. Sve se učitava preko funkcije load_data() u odgovarajuće promenljive. 

## Priprema podataka

In [5]:
print(train_dataset[0],train_labels[0]) #prikazuje se kako izgleda jedna recenzija
print("\n\nDuzina prve tri recenzije:")
print(len(train_dataset[0]),len(train_dataset[1]),len(train_dataset[2]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 1


Duzina prve tri recenzije:
218 1

Prikazali smo prvu recenziju u train setu, iako ne možemo trenutno da pročitamo vidimo da je pozitivna recenzija u pitanju.

Rekurentne neuronske mreže očekuju fiksnu dužinu sekvenci. Nakon jednostavne provere dužine recenzija vidimo da nisu sve iste. Potrebno je prepraviti trening i test set tako da  imaju fiksnu dužinu. Dužina recenzija treba da se odredi tako da gubi najmanje informacija u recenzijama, ali izbegava preduge sekvence.

In [6]:
#određujemo dužinu reči u recenziji
sequence_lengths = [len(seq) for seq in train_dataset]
maxlen = int(np.percentile(sequence_lengths, 65))
print(maxlen)

232


Ekspermentisanjem smo došli do najboljeg rezultata kada je dužina 232.

Potrebno je prepraviti trening i test set tako da se izbacuju sve dodatne reči ako je recenzija duža od 232 reči ili popunjavamo nulama ako je kraća od 232. Ovo je moguće uraditi preko pad_seqence funkcije.

In [7]:
REVIEW_MAX_LEN = maxlen

train_dataset = keras.preprocessing.sequence.pad_sequences(train_dataset,maxlen=REVIEW_MAX_LEN,padding='post')
test_dataset = keras.preprocessing.sequence.pad_sequences(test_dataset,maxlen=REVIEW_MAX_LEN,padding='post')

In [8]:
print(len(train_dataset[0]),len(train_dataset[1]),len(train_dataset[2]))

232 232 232


In [9]:
print(train_dataset.shape)
print(test_dataset.shape)

(25000, 232)
(25000, 232)


In [10]:
print(train_dataset[0])

[    1    14    22    16    43   530   973  1622  1385    65   458  4468
    66  3941     4   173    36   256     5    25   100    43   838   112
    50   670 22665     9    35   480   284     5   150     4   172   112
   167 21631   336   385    39     4   172  4536  1111    17   546    38
    13   447     4   192    50    16     6   147  2025    19    14    22
     4  1920  4613   469     4    22    71    87    12    16    43   530
    38    76    15    13  1247     4    22    17   515    17    12    16
   626    18 19193     5    62   386    12     8   316     8   106     5
     4  2223  5244    16   480    66  3785    33     4   130    12    16
    38   619     5    25   124    51    36   135    48    25  1415    33
     6    22    12   215    28    77    52     5    14   407    16    82
 10311     8     4   107   117  5952    15   256     4 31050     7  3766
     5   723    36    71    43   530   476    26   400   317    46     7
     4 12118  1029    13   104    88     4   381   

Dužine svih recenzija u trening i test setu su jednake.

U narednim blokovima definisaće se funkcije za šifrovanje i dešifrovanje recenzija koje će poslužiti za čitanje recenzija ili korišćenje modela da se predvide recenzije van skupa podataka. 

In [11]:
#Funkcija šifrovanja:

def encode_review(text):
    tokens=keras.preprocessing.text.text_to_word_sequence(text) 
    tokens=[vocabulary[word] if word in vocabulary else 0 for word in tokens]
    return keras.preprocessing.sequence.pad_sequences([tokens],REVIEW_MAX_LEN)[0]


Funkcija text_to_word_sequence radi tako što pretvara tekst iz parametra u listu reči (tokena). Na primer, rečenica "Ovo je primer." biće pretvorena u listu ["ovo", "je", "primer"]

Za svaku reč u listi tokena proverava da li se ta reč nalazi u rečniku 'vocabulary'. Ako se nalazi, uzima odgovarajući indeks te reči iz rečnika. Ako se ne nalazi, postavlja vrednost na 0.

I na kraju vraća se prvi član liste sekvenci, u ovom slučaju recenzije, koji sadrži indekse reči iz rečnika 'vocabulary' i dopunjava do 0 ako je to potrebno do recenzije dužine REVIEW_MAX_LEN.

In [12]:
#Provera da li radi funkcija za šifrovanje

tekst= "Movie was great, we had fun!"
encoded=encode_review(tekst)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  17  13  84  72  66 250]


In [13]:
#Funkcija dešifrovanja:

reverse_vocab = {value: key for (key, value) in vocabulary.items()} #inverzni rečnik

def decode_review(int_list):
    padding=0
    text=""
    for number in int_list:
        if number!=padding:
            text+=reverse_vocab[number]+" "
    return text[:-1]

Kreira se inverzni rečnik reverse_vocab, gde su ključevi originalni indeksi reči, a vrednosti su same reči. Ovo se koristi za mapiranje indeksa reči nazad u reči tokom dekodiranja.

Funkcija decode_review prima kao parametar listu indeksa reči tj. šifrovanu recenziju. Definišemo koji je dopunski karakter 'padding' i prazan string text u koji će se smeštati recenzija. Za svaki indeks iz liste koji nije padding dodaje odgovarajuću reč iz inverznog rečnika promenljivoj tekst. Na kraju, vraća dekodirani tekst bez poslednjeg razmaka.

In [14]:
#Provera da li radi funkcija za dešifrovanje

decoded=decode_review(encoded)
print(decoded)

movie was great we had fun


## Kreiranje modela

Za analizu sentimenata upotrebićemo model jednoslojnih rekurentnih neuronskih mreža (RNN). 

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

Sequential je model kojim se predstavlja sekvencijalni redosled slojeva u neuronskoj mreži. Svaki sloj u modelu je dodan jedan za drugim, a podaci prolaze kroz slojeve u određenom redosledu.

Embedding sloj mapira reči u vektore koji predstavljaju ulaz u sledeći sloj. NUM_WORDS označava ukupan broj reči u rečniku, a 32 označava dimenziju vektora.

LSTM (Long Short-Term Memory) je sloj koji se vezuje za rekurentne neuronske mreže (RNN) i predstavlja internu memoriju. Parametar označava broj neurona u sloju.

Dense je sloj u kojem je svaki neuron potpuno povezan sa svakim neuronom prethodnog sloja. Često se koristi za zadatak binarne klasifikacije. Parametar 1 predstavlja broj neurona koji su izlazi modela, a sigmoid aktivaciona funkcija se koristi kako bi se izlazi modela skalirali između 0 i 1, što je potrebno za binarnu klasifikaciju. 0 predstavlja negativnu recenziju, a 1 pozitivnu.

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

U ovom kodu radimo optimizaciju neuronske mreže tj. nameštamo ključne parametre za obuku modela. Parametar loss predstavlja funkciju gubitka "binary_crossentropy" se koristi kada imamo binarnu klasifikaciju. Za optimizaciju uzimamo Root Mean Square Propagation koji ima za cilj da minimizuje funkciju gubitka prilagođavanjem težina modela. Metriku kojom će evaluirati model biće 'accuracy' tj. tačnost. Tačnost predstavlja odnos tačno klasifikovanih (ispravnih predviđanja) u odnosu na ukupan broj predviđenih. 

In [18]:
history = model.fit(train_dataset,train_labels, epochs=10, validation_split=0.2)

Epoch 1/10


625/625 [==============================] - 53s 79ms/step - loss: 0.6359 - acc: 0.6072 - val_loss: 0.4685 - val_acc: 0.8066
Epoch 2/10
625/625 [==============================] - 52s 82ms/step - loss: 0.4230 - acc: 0.8399 - val_loss: 0.3782 - val_acc: 0.8600
Epoch 3/10
625/625 [==============================] - 51s 82ms/step - loss: 0.3349 - acc: 0.8842 - val_loss: 0.4720 - val_acc: 0.7980
Epoch 4/10
625/625 [==============================] - 53s 84ms/step - loss: 0.2903 - acc: 0.9049 - val_loss: 0.5247 - val_acc: 0.8260
Epoch 5/10
625/625 [==============================] - 52s 83ms/step - loss: 0.2483 - acc: 0.9179 - val_loss: 0.3582 - val_acc: 0.8706
Epoch 6/10
625/625 [==============================] - 52s 83ms/step - loss: 0.2264 - acc: 0.9248 - val_loss: 0.3549 - val_acc: 0.8590
Epoch 7/10
625/625 [==============================] - 52s 84ms/step - loss: 0.2038 - acc: 0.9352 - val_loss: 0.4603 - val_acc: 0.8378
Epoch 8/10
625/625 [==============================] - 53s 84

Treniranje modela. Broj epoha je 10 što znači da će ceo skup podataka biti prosleđen modelu tokom obuke 10 puta. Tokom svake epohe, model prolazi kroz ceo trening skup podataka i prilagođava svoje težine kako bi smanjio funkciju gubitka čime iterativno poboljšava model. 20% podataka će se koristiti za validaciju, dok će preostalih 80% biti korišćeno za samu obuku.

In [19]:
results = model.evaluate(test_dataset, test_labels)
print(results)

782/782 [==============================] - 22s 28ms/step - loss: 0.3873 - acc: 0.8630
[0.3873348832130432, 0.8629999756813049]


Model je tačan oko 85% slučajeva.

## Pedikcije

Testirajmo model nad primerima koji nisu u bazi. Pošto je potrebno prvo pripremiti nove podatke tako da mogu da se koriste u modelu, napisaćemo funkciju koja objediniti pripremu i predikciju.

In [20]:
def predict_review(text):
    encoded_text = encode_review(text)
    prediction = np.zeros((1,REVIEW_MAX_LEN))
    prediction[0] = encoded_text
    result = model.predict(prediction) 
    print(result[0])

In [21]:
review_pos = """
*level of enjoyment* #10 top tier - from start to finish The Dark Knight has you on the edge of your seat, enjoying every minute
*likelihood to recommend* #10 highest recommendation - if you haven't seen it, watch it, it is a masterpiece, the downside? likely to put all future Batman features to shame
*quality of acting* #9 excellent - Bale is the gold standard Batman, gruff, tough, aloof, positively fueled by the tragedies of his dark past. Ledger is the nightmare villain, twisted, evil, unapologetic, irrational, completely void of a moral compass
*quality of writing* #10 perfection - one of those movies you wish you could have a memory edit in order to watch again and again
*quality of intangibles* #8 excellent - visual is on point, dark gloomy Gotham, perfect setting for a hero to operate and succeed from the shadows very good - music and sound puts viewers in the mood and anticipating the next intense scene 
"""
predict_review(review_pos)


review_neg="""
Really, I could write a scathing review of this turd sandwich, but instead, I'm just going to be making a few observations and points I've deduced.
There's just no point in watching these movies anymore. Does any reader out there remember Scary Movie? Remember how it was original with a few comedic elements to it? There was slapstick, some funny lines, it was a pretty forgettable comedy, but it was worth the price of admission. Well, That was the last time this premise was funny. STOP MAKING THESE MOVIES. PLEASE.
I could call for a boycott of these pieces of monkey sh*t, but we all know there's going to be a line up of pre pubescent annoying little buggers, spouting crappy one liners like, "THIS IS SPARTA!" and, "IM RICK JAMES BITCH" so these movies will continue to make some form of monetary gain, considering the production value of this movie looks like it cost about 10 cents to make.
Don't see this movie. Don't spend any money on it. Go home, rent Airplane, laugh your ass off, and then silently judge the people that are talking about this movie on Monday. Do yourself a favor.
"""
predict_review(review_neg)

1/1 [==============================] - 1s 858ms/step
[0.9574349]
1/1 [==============================] - 0s 66ms/step
[0.01564289]
